In [2]:
import pandas as pd

In [3]:
data_ret = pd.read_csv("C:/Users/OS/OneDrive/Máy tính/BTC.csv")
data_label = pd.read_csv("D:/Download/BTCUSDT_label.csv")
label = data_label['LABEL']

In [4]:
# Add a row number column
data_ret['row_number'] = range(1, len(data_ret) + 1)

# Filter rows where the row number is divisible by 12 without remainder
data = data_ret[data_ret['row_number'] % 12 == 0]

In [16]:
import numpy as np
def get_return(data, time1):
    ret = data.copy()
    ret[f'return_{time1}'] = np.log(ret['CLOSE']/ret['CLOSE'].shift(time1))
    return ret

def get_volat(data, time2):
    volat = data.copy()
    volat = get_return(volat,1)
    volat[f"std_{time2}"] = volat[f'return_1'].rolling(window = time2).std()
    volat["std_long"] = volat[f'return_1'].rolling(window = 300).std()
    volat[f"volat_{time2}"] = volat[f"std_{time2}"]/volat["std_long"]
    return volat

In [18]:
import matplotlib.pyplot as plt
# Parameters
# drift coefficent

close_prices = data['CLOSE']
return1 = get_return(data,1)

mu = return1.mean()
# number of sims
M = 1000
# initial stock price
current_price = close_prices.iloc[-1]
target_price = current_price * np.exp(2 * mu)
# volatility
sigma = return1.std()

# Calculate the probability
probability = 1 - norm.cdf(np.log(target_price), np.log(current_price), sigma)
# Generate a series of probabilities for each time period
probabilities = []

for i in range(len(close_prices) - 2):
    current_price = close_prices.iloc[i]
    target_price = current_price * np.exp(2 * mu)
    probability = 1 - norm.cdf(np.log(target_price), np.log(current_price), sigma)
    probabilities.append(probability)
    
# Plot the series of probabilities
plt.plot(range(len(probabilities)), probabilities)
plt.xlabel("Time")
plt.ylabel("Probability")
plt.title("Probability of Current Price Reaching Next 2-hour Price")
plt.show()


D:\Anaconda\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


NameError: name 'norm' is not defined

In [7]:
import math

sequence = range(1, 27)
ret_dict = {}
for time1 in sequence:
    ret_dict[time1] = get_return(data, time1)

i = 22
w1 = 0.4
w2 = 0.25
w3 = 0.2
w4 = 0.1
w5 = 0.05
retur = ret_dict[i][f'return_{i}']*w1 + ret_dict[i+1][f'return_{i+1}']*w2 + ret_dict[i+2][f'return_{i+2}']*w3 + ret_dict[i+3][f'return_{i+3}']*w4 + ret_dict[i+4][f'return_{i+4}']*w5

time2 = 23
volat = get_volat(data,time2)

test = retur/(volat['volat_23']**15)
corrr = test.corr(label)
print(corrr)

0.01292444350925141


-0.0149888794183089
